In [66]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from newspaper import Article
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.snowball import EnglishStemmer
#from RecommendationEngine.py import simple_recommender

In [67]:
#First we want to import the particular article that we want
article = Article('https://www.cbc.ca/news/world/cocaine-bust-philadelphia-ship-1.5180447')
article.download()
article.parse()
article_text=article.text

In [68]:
#now we import all of the song Lyrics (this should probably be done in another python script)
df_songs = pd.DataFrame(pd.read_csv('lyrics.csv'))

In [69]:
df_songs.shape
#df_songs.head()

(362237, 6)

In [70]:
df_songs=df_songs.dropna(subset=['lyrics'])

In [71]:
df_songs.iloc[0,5]

"Oh baby, how you doing?\nYou know I'm gonna cut right to the chase\nSome women were made but me, myself\nI like to think that I was created for a special purpose\nYou know, what's more special than you? You feel me\nIt's on baby, let's get lost\nYou don't need to call into work 'cause you're the boss\nFor real, want you to show me how you feel\nI consider myself lucky, that's a big deal\nWhy? Well, you got the key to my heart\nBut you ain't gonna need it, I'd rather you open up my body\nAnd show me secrets, you didn't know was inside\nNo need for me to lie\nIt's too big, it's too wide\nIt's too strong, it won't fit\nIt's too much, it's too tough\nHe talk like this 'cause he can back it up\nHe got a big ego, such a huge ego\nI love his big ego, it's too much\nHe walk like this 'cause he can back it up\nUsually I'm humble, right now I don't choose\nYou can leave with me or you could have the blues\nSome call it arrogant, I call it confident\nYou decide when you find on what I'm working 

In [72]:
df_song_lyrics= list(df_songs.iloc[:,5])
df_song_lyrics[0]

"Oh baby, how you doing?\nYou know I'm gonna cut right to the chase\nSome women were made but me, myself\nI like to think that I was created for a special purpose\nYou know, what's more special than you? You feel me\nIt's on baby, let's get lost\nYou don't need to call into work 'cause you're the boss\nFor real, want you to show me how you feel\nI consider myself lucky, that's a big deal\nWhy? Well, you got the key to my heart\nBut you ain't gonna need it, I'd rather you open up my body\nAnd show me secrets, you didn't know was inside\nNo need for me to lie\nIt's too big, it's too wide\nIt's too strong, it won't fit\nIt's too much, it's too tough\nHe talk like this 'cause he can back it up\nHe got a big ego, such a huge ego\nI love his big ego, it's too much\nHe walk like this 'cause he can back it up\nUsually I'm humble, right now I don't choose\nYou can leave with me or you could have the blues\nSome call it arrogant, I call it confident\nYou decide when you find on what I'm working 

In [73]:
#now the at we have the songs in, we should drop all stop words, get rid of the /n, and drop any nans
i=0

for i in range(len(df_song_lyrics)):
    df_song_lyrics[i] = df_song_lyrics[i].replace('\n', ' ')

In [75]:
stemmer = EnglishStemmer()
analyzer = CountVectorizer().build_analyzer()

def stemmed_words(document):
    return (stemmer.stem(x) for x in analyzer(document))

In [ ]:
cv_stemmed=CountVectorizer(stop_words = 'english', analyzer=stemmed_words,min_df = 5, max_df=0.9)
cv_stemmed_lyrics=cv_stemmed.fit(df_song_lyrics)
cv_stemmed_lyrics=cv_stemmed.transform(df_song_lyrics)
cv_stemmed_lyrics.shape

In [ ]:
vectorizer = TfidfTransformer()
# tokenize and build vocab
vectorizer.fit(cv_stemmed_lyrics)
tf_idf_lyrics_vector = vectorizer.transform(cv_stemmed_lyrics)

In [ ]:
# summarize the big vector matrix
df_idf = pd.DataFrame(vectorizer.idf_, index=cv_stemmed.get_feature_names(),columns=["tf_idf_weights"])
print(tf_idf_lyrics_vector.shape)
print(df_idf.sort_values(by=['tf_idf_weights'])[::-1][0:20])
print(df_idf.sort_values(by=['tf_idf_weights'])[0:20])

In [107]:
#print(article_text)

Below we start to get the comparison to the article

In [115]:
cv_stemmed_article = cv_stemmed.transform(list(article_text))
tf_idf_article_vector = vectorizer.transform(cv_stemmed_article)
print(tf_idf_article_vector.shape)

(1712, 355234)


In [ ]:
print(tf_idf_article_vector)